In [1]:
import os
import pandas as pd
import glob
from pyspi.calculator import Calculator
from tqdm import tqdm

In [2]:
timings = pd.read_csv('/home/dan/data/connectivity/pyspi_testing/timings_output.csv')
# sort Time (s)
timings = timings.sort_values('Time (s)', ascending=True)
timings.to_csv('timings_sorted.csv',index=False)

In [3]:
timings

,Process,Time (s)
22,prec-sq_EllipticEnvelope,0.0002
26,prec-sq_MinCovDet,0.0002
25,prec-sq_LedoitWolf,0.0002
28,prec-sq_ShrunkCovariance,0.0002
27,prec-sq_OAS,0.0002
...,...,...
69,bary_softdtw_mean,5107.5567
53,dcorrx_maxlag-10,5521.5979
50,mgcx_maxlag-1,12781.9639
51,mgcx_maxlag-10,34912.1672


In [8]:
import re
def extract_time_and_counts(filepath, pre):
    """
    Extracts the time taken and counts from a summary table in the text.

    Args:
        filepath (str): Path to the input file containing the time and table.
        pre (str): Prefix to append to the dictionary keys.

    Returns:
        dict: A dictionary containing the time and counts, with default values as None.
    """


    
    result = {
        f"{pre} Successful": None,
        f"{pre} NaNs": None,
        f"{pre} Partial NaNs": None,
        f"{pre} Time Taken (s)": None,
    }

    try:
        # Read the text from the file
        with open(filepath, "r") as file:
            text = file.read()
    except Exception:
        return result

    # Extract the time taken in its own try-except block
    try:
        time_pattern = r"Time taken: (\d+\.\d+)s"
        time_match = re.search(time_pattern, text)
        if time_match:
            result[f"{pre} Time Taken (s)"] = float(time_match.group(1))
    except Exception:
        pass

    # Extract the counts in a separate try-except block
    try:
        counts_pattern = r"Successful\s*\|\s*(\d+)\s*\|\s*(\d+\.\d+)%.*?NaNs\s*\|\s*(\d+)\s*\|.*?Partial NaNs\s*\|\s*(\d+)"
        counts_match = re.search(counts_pattern, text, re.DOTALL)
        if counts_match:
            result[f"{pre} Successful"] = int(counts_match.group(1))
            result[f"{pre} NaNs"] = int(counts_match.group(3))
            result[f"{pre} Partial NaNs"] = int(counts_match.group(4))
    except Exception:
        pass

    return result

In [13]:
cpupath = '/home/dan/data/connectivity/pyspi_testing/timing/full_run'
cpupath = '/home/dan/data/connectivity/pyspi_testing/timing/timing_run_1'

pyspi_config_file = "/home/dan/data/connectivity/pyspi_testing/mymeasures.yaml"
pyspi_config_file = "/home/dan/data/connectivity/pyspi_testing/fast_config.yaml"
basecalc = Calculator(configfile=pyspi_config_file)

Loading configuration file: /home/dan/data/connectivity/pyspi_testing/fast_config.yaml
*** Importing module .statistics.basic
[0] Adding SPI .statistics.basic.Covariance(x,y,{'estimator': 'EmpiricalCovariance'})
Succesfully initialised SPI with identifier "cov_EmpiricalCovariance" and labels ['basic', 'unordered', 'linear', 'undirected', 'signed']
[1] Adding SPI .statistics.basic.Covariance(x,y,{'estimator': 'GraphicalLassoCV'})
Succesfully initialised SPI with identifier "cov_GraphicalLassoCV" and labels ['basic', 'unordered', 'linear', 'undirected', 'signed']
[2] Adding SPI .statistics.basic.Covariance(x,y,{'estimator': 'LedoitWolf'})
Succesfully initialised SPI with identifier "cov_LedoitWolf" and labels ['basic', 'unordered', 'linear', 'undirected', 'signed']
[3] Adding SPI .statistics.basic.Covariance(x,y,{'estimator': 'MinCovDet'})
Succesfully initialised SPI with identifier "cov_MinCovDet" and labels ['basic', 'unordered', 'linear', 'undirected', 'signed']
[4] Adding SPI .statis

In [14]:
rows = []
for measure in tqdm(basecalc._spis.keys()):
    
    cpu_result = extract_time_and_counts(os.path.join(cpupath, f'{measure}_run.out'), 'CPU')

    tmp = {'Measure': measure, }
    tmp.update(cpu_result)
    rows.append(tmp)

df = pd.DataFrame(rows)
df = df.sort_values(['CPU Successful','CPU NaNs','CPU Partial NaNs','CPU Time Taken (s)'],ascending=[False,True,True,True])
# reset the index
df = df.reset_index(drop=True)

100%|██████████| 136/136 [00:00<00:00, 27752.52it/s]


In [15]:
df.to_csv('timing_run_2~cpu_timing.csv')